# Experimenting with SQL

This notebook provides a light example for running an experiment for a natural language to SQL scenario.

## The Experiment

This experiment is simple, it appends static context to guide the LLM to generate SQL code from the NL prompt.
This can be seen in the [solution config file](./solution_configs/nl2sql_config.yaml).
It then calls the LLM and retrieves the first completion as output.

## Running on Azure ML vs Local

The current implementation runs the experiment on an Azure Machine Learning (AML) Workspace,
if you want to run this experiment locally, you need to create a new notebook using the [existing notebook template](../../experiments/templates/run_experiments_template.ipynb).

## Prerequisites

- You need to have the `sqlglot` package installed to run this experiment.
- You need to have an Azure subscription with an AML workspace.
- You need to have access to an OpenAI service.
- By default, this experiment uses the `code-davinci-002` model from OpenAI. If you do not have this model, please update the [solution config file](./solution_configs/nl2sql_config.yaml) to use the model you want.

In [ ]:
import os
import json
from pathlib import Path
import traceback
from ffmodel.core import orchestrator
from ffmodel.core.aml import aml_orchestrator

### Installing `sqlglot`

Before running this experiment, you need to install the `sqlglot` package if you
haven't already. Also, we're going to make a `requirements.txt` file for use
by AML to track the dependencies of this experiment.

First, let's create (or update) the `requirements.txt` file at the root of our
project:

In [ ]:
def prepare_requirements():
    path = "../../requirements.txt"
    requirements = "\n".join(
        [
            "# The following requirement is used by the nl2sql example:",
            "sqlglot~=11.5.8",
            "",  # Add a new line
        ]
    )

    # Create requirements.txt if needed
    if not os.path.exists("../../requirements.txt"):
        # Add sqlglot to requirements.txt
        with open("../../requirements.txt", "w") as f:
            f.write(requirements)
    else:
        # Check if sqlglot is in requirements.txt
        with open("../../requirements.txt", "r") as f:
            missing = not "sqlglot" in f.read()

        # Add sqlglot to requirements.txt
        if missing:
            with open("../../requirements.txt", "a") as f:
                f.write(f"\n{requirements}")


prepare_requirements()

Now let's install our dependencies:

In [ ]:
%pip install -r ../../requirements.txt

## Configs

This section captures the experimentation configs.

- `experiment_name`: the name of this experiment
- `solution_configs`: an array that holds the paths to the solution configuration yaml files describing solutions that we'd like to experiment with
- `environment_config_path`: the path to an environment configuration yaml. Follow the instructions captured [here](../../docs/guides/environment_configs.md).
- `experiment_output_path`: the path to store the output from all experiments

In [ ]:
experiment_name = "nl2sql"
solution_configs = ["solution_configs/nl2sql_config.yaml"]
environment_config_path = "~/.ffmodel"
experiment_output_path = os.path.join("outputs", experiment_name)

## Helper functions

Using the FFModel orchestrator to run the experiment on AzureML

In [ ]:
def run_ffmodel_experiment(solution_config_path):
    """
    Function that runs an FFModel experiments within AML. The resulting job name will be mapped to it's
    corresponding exp config file and will be returned as output
    """
    global aml_jobs

    print(f"Executing solution config file {solution_config_path}")
    print("----------------------------------------------")

    # execute experiment
    aml_job_name = orchestrator.execute_experiment_on_aml(solution_config_path, environment_config_path)

    print(f"Solution config {solution_config_path} - with job name {aml_job_name}")
    aml_jobs[aml_job_name] = {"solution_config": solution_config_path}


def get_data_models(aml_jobs: dict) -> dict:
    """
    This function waits until all experiments finished executing and then downloads their outputs.
    """
    for aml_job_name in aml_jobs:
        # wait for the experiment to finish running
        aml_orchestrator.wait_for_completion(aml_job_name)
        # retrieve data model once the job is completed
        data_models = aml_orchestrator.retrieve_final_data_models(aml_job_name)
        aml_jobs[aml_job_name]["data_models"] = data_models

    return aml_jobs

## Experimentation

Run the experiments and capture their outputs.

In [ ]:
# Global variables
aml_jobs = {}
aml_job_outputs = {}

# ensure the outputs path exists before running any experiment
Path(experiment_output_path).mkdir(parents=True, exist_ok=True)

print(f"Running the following experiments: \n {str(solution_configs)}")
try:
    for exp in solution_configs:
        run_ffmodel_experiment(exp)
except:
    traceback.print_exc()

## Download the experiment outputs

Both experiment pipelines are running on AML. At this time, we need to wait for them to finish executing before we can compare the experiments.

In [ ]:
aml_jobs = get_data_models(aml_jobs)

for job in aml_jobs:
    output_path = os.path.basename(aml_jobs[job]["solution_config"])
    output_path = os.path.splitext(output_path)[0]
    output_path = f"{os.path.join(experiment_output_path, output_path)}.jsonl"
    with open(output_path, "w") as f:
        f.write("\n".join(json.dumps(item.to_dict()) for item in aml_jobs[job]["data_models"]))